In [1]:
import pandas as pd

## All information is randomly generated, contains no PII whatsoever
## This combines all relevant data (from different tables) a manual user would use to identify a payment, puts it in a single row per client
## (Additional data like real-time approx. pending balance can be used to further confirm payments)

# Load each CSV file into a dataframe
df_bank = pd.read_csv('csv/01-bank.csv')
df_client = pd.read_csv('csv/02-client.csv')
df_student = pd.read_csv('csv/03-student.csv')

# Additional transformations/clean-up would go here. Assigned column names, replacing NaN values, cleaning up accents, etc.

# Merge the clients and students dataframes
df_client_combined = pd.merge(df_client, df_student, left_on='client id', right_on='associated client id')

# Put all student info in the same row as clients with a pivot table
df_client_combined = df_client_combined.pivot_table(
    index=['client id', 'name', 'last name', 'email1', 'email2', 'handle', 'account number'],
    columns=df_client_combined.groupby("client id").cumcount()+1,
    values=['student name', 'student last name', 'grade'],
    aggfunc='first'
).reset_index()

# Flatten pivot table, for a clean searchable index with one client per row.
df_client_combined.columns = df_client_combined.columns.to_series().apply(
    lambda x: ' '.join(str(y) for y in x if y).strip()
)

# Print the dataframe
df_client_combined
#print(df_client_combined.columns)
## Data ready to be uploaded/indexed/searched by elasticsearch

,client id,name,last name,email1,email2,handle,account number,grade 1,grade 2,grade 3,student last name 1,student last name 2,student last name 3,student name 1,student name 2,student name 3
0,33111,Karina,Weeks,KWeeks@gmail.com,KWeeks@hotmail.com,KWeeks,ES408636776,4,8,9,Weeks,Weeks,Weeks,Jaylan,Jaylen,Yandel
1,33112,Anders,Klein,AKlein@gmail.com,AKlein@hotmail.com,AKlein,ES137237055,8,5,6,Klein,Klein,Klein,Rebecca,Benjamin,Camilla
2,33113,Elianna,Evans,EEvans@gmail.com,EEvans@hotmail.com,EEvans,ES719897276,1,2,12,Evans,Evans,Evans,Draven,Destinee,Fiona
3,33114,Elias,Cabrera,ECabrera@gmail.com,ECabrera@hotmail.com,ECabrera,ES909374860,12,9,11,Cabrera,Cabrera,Cabrera,Makai,Jamya,Deborah
4,33115,Daleyza,Richard,DRichard@gmail.com,DRichard@hotmail.com,DRichard,ES251352145,5,9,1,Richard,Richard,Richard,Alayna,Titus,Azaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,33206,Sullivan,Gaines,SGaines@gmail.com,SGaines@hotmail.com,SGaines,ES388224780,11,8,7,Gaines,Gaines,Gaines,Tamara,Chana,Daniela
89,33207,Aya,Pugh,APugh@gmail.com,APugh@hotmail.com,APugh,ES177268945,1,4,3,Pugh,Pugh,Pugh,Bryson,Jude,Charlie
90,33208,Judson,Joseph,JJoseph@gmail.com,JJoseph@hotmail.com,JJoseph,ES816342319,7,10,1,Joseph,Joseph,Joseph,Christine,Jacob,Albert
91,33209,Gracelynn,Quinn,GQuinn@gmail.com,GQuinn@hotmail.com,GQuinn,ES873141612,4,2,5,Quinn,Quinn,Quinn,Ezra,Laurel,Jermaine
